In [1]:
import os
from google.cloud import storage, bigquery
import pandas as pd
from xgboost import XGBRegressor
from datetime import datetime
import time
import requests
from urllib.parse import quote
import numpy as np
from sklearn.neighbors import BallTree
import json

### Download these files to pull from google cloud
# pip install google-cloud-bigquery
# pip install db-dtypes
# pip install google-cloud-bigquery-storage

# --- Input ---
input_dict = {
    'month': 4,
    'year': 2029,
    'flat_type': '2 ROOM',
    'town': 'ANG MO KIO/ALJUNIED',
    'block': '108',
    'street_name': 'ANG MO KIO AVE 4',
    'storey_range': '13 TO 15',
    'floor_area_sqm': 48.0,
    'flat_model': 'New Generation',
    'lease_commence_date': 1990,
    'remaining_lease': '62 years 09 months'
}

# --- Step 1: Download Model from GCS ---
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "is3107-project-457501-4f502924c0f9.json"
gcs_client = storage.Client()
bucket = gcs_client.bucket("is3107-project-bucket")
model_blob = bucket.blob("models/hdb_xgb_model.json")
model_blob.download_to_filename("hdb_xgb_model.json")
model_columns_blob = bucket.blob("models/model_columns.json")
model_columns_blob.download_to_filename("model_columns.json")

# --- Download Stocks Data ---
stocks_blob = bucket.blob("datasets/stock.csv")
stocks_blob.download_to_filename("stock.csv")

# --- Download Unemployment Data ---
unemp_blob = bucket.blob("datasets/unemployment.csv")
unemp_blob.download_to_filename("unemployment.csv")

# --- Download clustered school and mrt data ---
mrt_blob = bucket.blob("temporary/mrt_clustered.csv")
mrt_blob.download_to_filename("mrt_clustered.csv")
schools_blob = bucket.blob("temporary/schools_clustered.csv")
schools_blob.download_to_filename("schools_clustered.csv")

# Load model and datasets
model = XGBRegressor()
model.load_model("hdb_xgb_model.json")
df_stocks = pd.read_csv("stock.csv")
df_unemp = pd.read_csv("unemployment.csv")
df_mrt = pd.read_csv("mrt_clustered.csv")
df_schools = pd.read_csv("schools_clustered.csv")

# --- Token & Geocoding ---
def get_token():
    url = "https://www.onemap.gov.sg/api/auth/post/getToken"
    payload = {"email": "Jevan.Koh@u.nus.edu", "password": "JekoP4ssw0rd@123"}
    response = requests.post(url, json=payload)
    return response.json().get("access_token") if response.status_code == 200 else None

def get_lat_lon(search_val):
    token = get_token()
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={quote(search_val)}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": f"Bearer {token}"}
    try:
        response = requests.get(url, headers=headers)
        results = response.json().get("results", [])
        if results:
            return float(results[0]["LATITUDE"]), float(results[0]["LONGITUDE"]), results[0].get("POSTAL")
    except:
        pass
    return None, None, None

# --- Data Preparation ---
df = pd.DataFrame([input_dict])
df['years_from_lease'] = datetime.now().year - df['lease_commence_date']
years, months = map(int, df['remaining_lease'].str.extract(r'(\d+)\s+year[s]?\s+(\d+)\s+month[s]?').iloc[0])
df['remaining_lease_months'] = years * 12 + months
df.drop(columns=['remaining_lease'], inplace=True)
df['address'] = df['block'].str.strip() + ' ' + df['street_name'].str.strip()
df.drop(columns=['block', 'street_name'], inplace=True)
lat, lon, postal = get_lat_lon(df['address'].iloc[0])
df['latitude'] = lat
df['longitude'] = lon

# --- Add macro indicators ---
df['Average_Close'] = df_stocks.loc[
    (df_stocks['Year'] == df.at[0, 'year']) & (df_stocks['Month'] == df.at[0, 'month']),
    'Average_Close'].values[0]

df['Unemployment Rate'] = df_unemp.loc[
    (df_unemp['Year'] == df.at[0, 'year']) & (df_unemp['Month'] == df.at[0, 'month']),
    'Unemployment Rate'].values[0]

# --- Rename macro cols ---
df = df.rename(columns={'Average_Close': 'average_close', 'Unemployment Rate': 'unemployment_rate'})

# --- Preprocess school clusters ---
df_schools = df_schools.rename(columns={'cluster': 'school_cluster'})
df_mrt = df_mrt.rename(columns={'STATION_NA': 'station_name', 'cluster_25': 'mrt_cluster'})
df_schools['primary_schoool_count'] = (df_schools['mainlevel_code'] == 'PRIMARY').astype(int)
cluster_centroids = df_schools.groupby('school_cluster').agg({
    'cluster_center_lat': 'first',
    'cluster_center_lng': 'first',
    'school_count': 'first',
    'sap_ind_pct': 'mean',
    'autonomous_ind_pct': 'mean',
    'gifted_ind_pct': 'mean',
    'ip_ind_pct': 'mean',
    'primary_schoool_count': 'sum'
}).reset_index()

# --- Nearest school cluster ---
df_coords = np.radians(df[['latitude', 'longitude']])
cluster_coords = np.radians(cluster_centroids[['cluster_center_lat', 'cluster_center_lng']])
school_tree = BallTree(cluster_coords, metric='haversine')
school_dist, school_idx = school_tree.query(df_coords, k=1)
df['nearest_school_cluster'] = cluster_centroids.iloc[school_idx.flatten()]['school_cluster'].values
df = df.merge(cluster_centroids, left_on='nearest_school_cluster', right_on='school_cluster', how='left')
df['distance_to_nearest_school'] = school_dist.flatten() * 6371000

df.drop(columns=['cluster_center_lat', 'cluster_center_lng', 'school_cluster'], inplace=True)

# --- Nearest MRT cluster ---
mrt_centroids = df_mrt.groupby('mrt_cluster').agg({
    'cluster_lat': 'first',
    'cluster_long': 'first'
}).reset_index()
mrt_coords = np.radians(df_mrt[['cluster_lat', 'cluster_long']])
mrt_tree = BallTree(mrt_coords, metric='haversine')
mrt_dist, mrt_idx = mrt_tree.query(df_coords, k=1)
df['nearest_mrt_cluster'] = mrt_centroids.iloc[mrt_idx.flatten()]['mrt_cluster'].values
df = df.merge(mrt_centroids, left_on='nearest_mrt_cluster', right_on='mrt_cluster', how='left')
df['distance_to_nearest_mrt'] = mrt_dist.flatten() * 6371000
df.drop(columns=['cluster_lat', 'cluster_long', 'mrt_cluster'], inplace=True)

# --- Feature Engineering ---
df['flat_age_at_resale'] = df['year'] - df['lease_commence_date']
df['storey_mean'] = df['storey_range'].str.extract(r'(\d+)\s+TO\s+(\d+)').astype(float).mean(axis=1)
df['is_high_floor'] = (df['storey_mean'] > 12).astype(int)
df['is_big_unit'] = (df['floor_area_sqm'] > 110).astype(int)
df['school_quality'] = df[['sap_ind_pct', 'autonomous_ind_pct', 'gifted_ind_pct', 'ip_ind_pct']].sum(axis=1)

# --- Region Mapping ---
df['town'] = df['town'].str.replace('/', '_')
town_to_region = {
    "ANG MO KIO": "Rest of Central Region",
    "KALLANG_WHAMPOA": "Rest of Central Region",
    # (include all others as needed...)
}
df['region'] = df['town'].str.upper().map(lambda x: town_to_region.get(x, 'Others'))

# --- One-hot encode & align ---
df.drop(columns=['address', 'storey_range'], inplace=True)
df_encoded = pd.get_dummies(df, columns=['region', 'town', 'flat_type', 'flat_model'], drop_first=False)
with open("model_columns.json", "r") as f:
    model_columns = json.load(f)
df_encoded = df_encoded.reindex(columns=model_columns, fill_value=0)
df.drop(columns=['region'], inplace=True)

# --- Prediction ---
predicted_price = model.predict(df_encoded)[0]
print(f"\n💰 Predicted resale price: ${predicted_price:,.2f}")


💰 Predicted resale price: $385,076.56
